# Compare contrails with satellite

- Authors: Marc Shapiro, Zeb Engberg
- Date: 2023-04-14
- `pycontrails`: v0.39.6

End-to-end analysis using `pycontrails` and sample data.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/contrailcirrus/2023-04-pycontrails-workshop/blob/main/notebooks/04-Full-Analysis.ipynb)

## Goals

### Load Data

1. Load pre-processed flight data from 2020-01-01
3. Select flight trajectories in a spatial domain of interest (e.g. within a GOES image)
2. Load meteorology data from ECMWF for the waypoints in this space

### Compare contrail output with satellite imagery

1. Calculate contrail evolution using `CoCiP`
2. Plot contrails on top of GOES imagery

## Download data

In [42]:
# GCP Bucket with sample data
GCP_BUCKET = "gs://2023-04-pycontrails-workshop"

# make data/ directory
!mkdir -p data/flight/global

# sync all data files locally (~15 GB)
!gsutil -m rsync -r {GCP_BUCKET}/data/flight/global data/flight/global
!gsutil -m rsync -r {GCP_BUCKET}/data/met data/met
!gsutil -m rsync -r {GCP_BUCKET}/data/rad data/rad


both the source and destination. Your crcmod installation isn't using the
module's C extension, so checksumming will run very slowly. If this is your
first rsync since updating gsutil, this rsync can take significantly longer than
usual. For help installing the extension, please see "gsutil help crcmod".

Building synchronization state...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Starting synchronization...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that 

## Load sample flight data

Load flight data from the `data/flights/sample` directory

In [76]:
import pandas as pd

In [81]:
metadata = pd.read_parquet("data/flight/sample/flight-metadata.pq")

waypoints = pd.read_parquet("data/flight/sample/flight-waypoints.pq")
waypoints.head()

,flight_id,longitude,latitude,altitude_ft,time,time_unix
0,200101-10205-CSN3281,110.039553,25.219828,570.000000,2020-01-01 00:15:05,1577837705
1,200101-10205-CSN3281,110.043064,25.291929,5131.461402,2020-01-01 00:16:35,1577837795
2,200101-10205-CSN3281,110.044822,25.327980,5200.000000,2020-01-01 00:17:04,1577837824
3,200101-10205-CSN3281,110.046580,25.364030,5200.000000,2020-01-01 00:17:32,1577837852
4,200101-10205-CSN3281,110.018589,25.402096,7050.000000,2020-01-01 00:18:12,1577837892


Find waypoints in region of GOES image

In [78]:
df = waypoints.loc[
    (waypoints["longitude"] > -90.0) & \
    (waypoints["longitude"] < -67.5) & \
    (waypoints["latitude"] > 0) & \
    (waypoints["latitude"] < 21)
]

In [79]:
df.sort_values(by="time", ascending=False).head(5)

,flight_id,longitude,latitude,altitude_ft,time,time_unix
19578,200101-30368-LAN500,-79.088828,20.956095,37975.0,2020-01-01 10:05:00,1577873100
19577,200101-30368-LAN500,-79.101075,20.823196,38000.0,2020-01-01 10:04:02,1577873042
19576,200101-30368-LAN500,-79.109546,20.731544,38000.0,2020-01-01 10:03:22,1577873002
19575,200101-30368-LAN500,-79.118007,20.639892,38000.0,2020-01-01 10:02:42,1577872962
19574,200101-30368-LAN500,-79.126458,20.548238,38000.0,2020-01-01 10:02:02,1577872922


In [80]:
df

,flight_id,longitude,latitude,altitude_ft,time,time_unix
7386,200101-2131-VIR66G,-77.913399,18.503700,4.000000,2020-01-01 01:37:57,1577842677
7387,200101-2131-VIR66G,-77.886946,18.540995,2891.805828,2020-01-01 01:38:51,1577842731
7388,200101-2131-VIR66G,-77.860483,18.578287,5610.191686,2020-01-01 01:39:36,1577842776
7389,200101-2131-VIR66G,-77.834007,18.615574,8045.347608,2020-01-01 01:40:11,1577842811
7390,200101-2131-VIR66G,-77.781022,18.690137,12095.617175,2020-01-01 01:41:12,1577842872
...,...,...,...,...,...,...
45848,200101-750-CMP277,-79.843623,0.325790,39000.000000,2020-01-01 03:50:41,1577850641
45849,200101-750-CMP277,-79.844387,0.247558,39000.000000,2020-01-01 03:51:21,1577850681
45850,200101-750-CMP277,-79.845150,0.169326,39000.000000,2020-01-01 03:52:01,1577850721
45851,200101-750-CMP277,-79.845913,0.091094,39000.000000,2020-01-01 03:52:41,1577850761


Select a single flight by index

In [8]:
flight = waypoints.loc[waypoints["flight_id"] == "200101-30368-LAN500"].reset_index(drop=True)
flight.head()

,flight_id,longitude,latitude,altitude_ft,time,time_unix
0,200101-30368-LAN500,-70.785797,-33.393002,1555.00,2020-01-01 02:59:07.163,1577847547
1,200101-30368-LAN500,-70.783429,-33.434052,2850.00,2020-01-01 03:00:02.000,1577847602
2,200101-30368-LAN500,-70.820408,-33.453238,3743.75,2020-01-01 03:00:42.000,1577847642
3,200101-30368-LAN500,-70.857404,-33.472413,4637.50,2020-01-01 03:01:22.000,1577847682
4,200101-30368-LAN500,-70.894416,-33.491577,5531.25,2020-01-01 03:02:02.000,1577847722


## Load global flight data

In [48]:
import pandas as pd

from pycontrails import Flight, Fleet
from pycontrails.physics import units

In [ ]:
# Read metadata extracted for this day 2020-01-01
metadata = pd.read_parquet("data/flight/global/20200101-summary.pq")
metadata.set_index("flight_id", inplace=True)

In [90]:
hours = range(10, 13)

waypoint_frames = []
for hr in hours:
    
    # Read one hour (2020-01-01 12:00:00Z) of cleaned global flight data
    waypoint_frames.append(pd.read_parquet(f"data/flight/global/20200101T{str(hr).zfill(2)}.pq"))
    
# aggregate all waypoints into a single dataframe
waypoints = pd.concat(waypoint_frames)

restrict to within a single goes image

In [91]:
df = waypoints.loc[
    (waypoints["longitude"] > -90.0) & \
    (waypoints["longitude"] < -67.5) & \
    (waypoints["latitude"] > 0) & \
    (waypoints["latitude"] < 21)
]

In [92]:
# 119 unique flights in this region during this hour
len(df["flight_id"].unique())

280

Load waypoints into a list of `Flight` objects

The flight data necessary to run CoCiP is:

- `flight_id` - str or int
- `longitude` - WGS84
- `latitude` - WGS84
- `altitude` - meters

To run CoCiP without using an aircraft performance model (e.g. BADA), you need the additional data:

- `engine_efficiency`
- `fuel_flow` - kg s**-1
- `aircraft_mass` - kg
- `nvpm_ei_n` - # kg**-1
- `wingspan` - meters

Here we will grab the pre-computed aircraft performance to avoid running processing emissions

In [93]:
# columns required to run cocip without aircraft performance model
## TODO: Remove other columns from data
cols = ["flight_id", "longitude", "latitude", "altitude", "time", 
        "engine_efficiency", "fuel_flow", "aircraft_mass", "nvpm_ei_n"]

# init empty list of Flight classes
flights = []

# load metadata and create Flight class
for fid, fl in df.groupby("flight_id"):
    # sort on time
    fl = fl.sort_values(by="time")
    
    # Aircraft and engine properties
    attrs = {
        "flight_id": fid,
        "wingspan": metadata.loc[fid]["wingspan"],

        # not necessary for running the model in this configuration
        "aircraft_type": metadata.loc[fid]["aircraft_type_icao"],
    }

    # Create flight object
    fl = Flight(data=fl[cols], attrs=attrs)
    flights.append(fl)

## Load data from ECMWF

In [94]:
import xarray as xr

from pycontrails import MetDataset

### Load from staged files

- NetCDF files in `data/met` contain pressure level data for all required `CoCiP` meteorology variables.
- NetCDF files in `data/rad` contain single level data for all required `CoCiP` radiation variables.

In [55]:
ds_pressure_levels = xr.open_mfdataset("data/met/*.nc")
ds_single_level = xr.open_mfdataset("data/rad/*.nc")

In [56]:
met = MetDataset(ds_pressure_levels)

# pycontrails expects a level dimension for all MetDatasets (even single level)
# we've adopted the convention to add a level=-1 coordinate to single level data
ds_single_level = ds_single_level.expand_dims({'level': [-1]})
rad = MetDataset(ds_single_level)

In [57]:
# update variables to be in the required format
# see https://py.contrails.org/api/pycontrails.models.cocip.Cocip.html#pycontrails.models.cocip.Cocip
met.data = met.data.rename({
    "z": "geopotential",
    "ciwc": "specific_cloud_ice_water_content",
    "q": "specific_humidity",
    "t": "air_temperature",
    "u": "eastward_wind",
    "v": "northward_wind",
    "w": "lagrangian_tendency_of_air_pressure"
})

rad.data = rad.data.rename({
    "tsr": "toa_net_upward_shortwave_flux",
    "ttr": "toa_outgoing_longwave_flux"
})

In [58]:
# convert ERA5 radiation values from accumulated values (J m**-2) 
# to average instantanesous values (W m**-2)
dt_accumulation = 60 * 60
rad.data["toa_net_upward_shortwave_flux"] = rad.data["toa_net_upward_shortwave_flux"] / dt_accumulation
rad.data["toa_outgoing_longwave_flux"] = rad.data["toa_outgoing_longwave_flux"] / dt_accumulation

### Use pycontrails ERA5 interface

Automates the processing above

In [95]:
import pathlib

from pycontrails.datalib.ecmwf import ERA5

#### Set domain

This is always required, even if the file only contains the dimensions / variables necessary.

In [96]:
# set time domain from the first flight waypoint
# through 10 hours past the last flight waypoint
time = (flights[0]["time"].min(), flights[-1]["time"].min() + pd.Timedelta("10H"))

# select variables by standard name
pressure_level_variables = [
    "air_temperature",
    "specific_humidity",
    "eastward_wind",
    "northward_wind",
    "lagrangian_tendency_of_air_pressure",
    "specific_cloud_ice_water_content",
    "geopotential",
]

single_level_variables = [
    "top_net_solar_radiation",
    "top_net_thermal_radiation"
]

# select pressure levels from meteorology
pressure_levels = [
    1000,
    975,
    950,
    925,
    900,
    875,
    850,
    825,
    800,
    775,
    750,
    700,
    650,
    600,
    550,
    500,
    450,
    400,
    350,
    300,
    250,
    225,
    200,
    175,
    150,
    125,
    100,
]

In [97]:
# Load ERA5 data from local files
era5pl = ERA5(
    time=time,
    variables=pressure_level_variables,
    pressure_levels=pressure_levels,
    paths="data/met/*.nc",
    cachestore=None,
)
era5sl = ERA5(
    time=time,
    variables=single_level_variables,
    paths="data/rad/*.nc",
    cachestore=None,
)

# Open MetDataset from sources
met = era5pl.open_metdataset(xr_kwargs=dict(parallel=False))
rad = era5sl.open_metdataset(xr_kwargs=dict(parallel=False))

This next cell is unnecessary since we've already staged the data above. 

This would allow you to download new domains automatically

In [ ]:
# Download and cache ERA5 files for this domain
# Note you must have an account with CDS for this to work
# Provide API credentials in the inputs below
era5pl = ERA5(
    time=time,
    variables=pressure_level_variables,
    pressure_levels=pressure_levels,
    # url="https://cds.climate.copernicus.eu/api/v2",
    # key="<key>"
)
era5sl = ERA5(
    time=time,
    variables=single_level_variables,
    # url="https://cds.climate.copernicus.eu/api/v2",
    # key="<key>"
)

# Open MetDataset from sources
# This is the step where the data is downloaded
met = era5pl.open_metdataset(xr_kwargs=dict(parallel=False))
rad = era5sl.open_metdataset(xr_kwargs=dict(parallel=False))

## Run CoCiP model

In [98]:
import numpy as np
from pycontrails.models.cocip import Cocip
from pycontrails.models.humidity_scaling import ExponentialBoostHumidityScaling

Customize model params.
See [CocipParams](https://py.contrails.org/api/pycontrails.models.cocip.CocipParams.html#pycontrails.models.cocip.CocipParams) for a list of possible parameters.

In [99]:
# create a humidity correction based on Teoh 2022
humidity_scaling = ExponentialBoostHumidityScaling()

In [100]:
params = {
    "dt_integration": np.timedelta64(5, "m"),
    "humidity_scaling": humidity_scaling,
    "process_emissions": False
}

In [101]:
cocip = Cocip(met, rad, params)

In [102]:
%%time
output = cocip.eval(flights)

/Users/marcshapiro/computing/contrailcirrus/pycontrails/pycontrails/models/cocip/cocip.py:585: UserWarning: Flight trajectory has segment lengths close to or exceeding the 'max_seg_length_m' parameter. Evolved contrail segments may reach their end of life artificially early. Either resample the flight with the 'resample_and_fill' method (recommended), or use a larger 'max_seg_length_m' parameter. Current values: max_seg_length_m=40000.0, max_seg_length_on_trajectory=302915.79276262
  warnings.warn(
/Users/marcshapiro/computing/contrailcirrus/pycontrails/pycontrails/models/cocip/contrail_properties.py:444: UserWarning: All tau_contrail values are nan. This may be due to waypoints all lying outside of the met interpolation grid. It could indicate an issue with interpolation, or an insufficient met domain.
  warnings.warn(


CPU times: user 39.9 s, sys: 15 s, total: 54.9 s
Wall time: 20.8 s


In [103]:
# retrieve the contrail output
contrails = cocip.contrail

In [105]:
contrails.sort_values("time")

,waypoint,flight_id,formation_time,time,age,longitude,latitude,altitude,level,continuous,...,tau_contrail,dn_dt_agg,dn_dt_turb,rf_sw,rf_lw,rf_net,persistent,ef,timestep,age_hours
0,97,200101-32463-XADOC,2020-01-01 12:00:04,2020-01-01 12:05:00,0 days 00:04:56,-80.204678,16.699380,11883.252865,196.892221,True,...,0.005510,5.873845e-17,0.000106,-2.018852,0.401758,-1.617094,True,-9.756374e+08,0,0.082222
23,101,200101-770-CMP295,2020-01-01 12:04:15,2020-01-01 12:05:00,0 days 00:00:45,-80.193887,12.514752,11841.264678,198.200194,True,...,0.104618,2.714789e-18,0.000155,-7.462571,2.527935,-4.934636,True,-1.230436e+08,0,0.012500
22,100,200101-770-CMP295,2020-01-01 12:03:35,2020-01-01 12:05:00,0 days 00:01:25,-80.209266,12.593991,11841.393848,198.196157,True,...,0.024592,1.888470e-19,0.000103,-3.062054,0.317066,-2.744988,True,-2.201574e+08,0,0.023611
21,139,200101-75422-AAL932,2020-01-01 12:01:16,2020-01-01 12:05:00,0 days 00:00:00,-80.300452,12.616942,11530.733151,208.147164,False,...,0.048225,2.835006e-18,0.000053,-4.949975,1.476515,-3.473460,True,0.000000e+00,0,0.000000
20,138,200101-75422-AAL932,2020-01-01 12:00:36,2020-01-01 12:05:00,0 days 00:04:24,-80.291620,12.533083,11531.666126,208.116543,True,...,0.173016,8.176811e-18,0.000047,-7.635602,6.927424,-0.708178,True,4.479996e+07,0,0.073333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,135,200101-33241-IBE6461,2020-01-01 21:49:06,2020-01-02 01:00:00,0 days 03:10:54,-76.921382,2.680694,12080.805476,190.853170,True,...,0.143579,4.226815e-17,0.000004,0.000000,NaN,NaN,True,NaN,155,3.181667
60,136,200101-33241-IBE6461,2020-01-01 21:50:02,2020-01-02 01:00:00,0 days 03:09:58,-77.008842,2.583624,12050.250905,191.774948,True,...,0.153415,3.708209e-17,0.000004,0.000000,NaN,NaN,True,NaN,155,3.166111
61,137,200101-33241-IBE6461,2020-01-01 21:50:42,2020-01-02 01:00:00,0 days 03:09:18,-77.067030,2.505416,12029.695891,192.397561,True,...,0.186529,4.201056e-17,0.000004,0.000000,NaN,NaN,True,NaN,155,3.155000
53,129,200101-33241-IBE6461,2020-01-01 21:45:06,2020-01-02 01:00:00,0 days 03:14:54,-76.647595,3.128936,12015.733964,192.821621,True,...,0.132020,6.942489e-17,0.000005,0.000000,NaN,NaN,True,NaN,155,3.248333
